In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("AladinReview") \
    .master("spark://namenode:7077") \
    .config("spark.executor.cores", "2") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.instances", "3") \
    .config("spark.jars.packages", "org.mongodb.spark:mongo-spark-connector_2.12:3.0.1") \
    .config("spark.mongodb.read.connection.uri", "mongodb://client:27017") \
    .config("spark.mongodb.read.database", "bookdb") \
    .config("spark.mongodb.read.collection", "reviews") \
    .getOrCreate()

:: loading settings :: url = jar:file:/home/hadoop/miniconda3/lib/python3.12/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
org.mongodb.spark#mongo-spark-connector_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-3cec7a2d-0f95-42ff-a8d6-9be49f130984;1.0
	confs: [default]
	found org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 in central
	found org.mongodb#mongodb-driver-sync;4.0.5 in central
	found org.mongodb#bson;4.0.5 in central
	found org.mongodb#mongodb-driver-core;4.0.5 in central
:: resolution report :: resolve 124ms :: artifacts dl 3ms
	:: modules in use:
	org.mongodb#bson;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-core;4.0.5 from central in [default]
	org.mongodb#mongodb-driver-sync;4.0.5 from central in [default]
	org.mongodb.spark#mongo-spark-connector_2.12;3.0.1 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts 

In [2]:
spark

In [ ]:
df_temp = spark.read.format("mongo") \
    .option("uri", "mongodb://client:27017/") \
    .option("database", "bookdb") \
    .option("collection", "reviews") \
    .load()

df_temp.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- book_id: string (nullable = true)
 |-- content: string (nullable = true)
 |-- date: string (nullable = true)
 |-- likes: integer (nullable = true)
 |-- rating: integer (nullable = true)
 |-- review_type: string (nullable = true)
 |-- title: string (nullable = true)
 |-- user: string (nullable = true)



In [ ]:
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

schema = StructType([
    StructField("_id", StructType([
        StructField("oid", StringType(), True)
    ]), True),
    StructField("book_id", StringType(), True),
    StructField("content", StringType(), True),
    StructField("date", StringType(), True),
    StructField("likes", IntegerType(), True),
    StructField("rating", IntegerType(), True),
    StructField("review_type", StringType(), True),
    StructField("title", StringType(), True),
    StructField("user", StringType(), True)
])

reviews_df = spark.read.format("mongo") \
    .option("uri", "mongodb://client:27017/") \
    .option("database", "bookdb") \
    .option("collection", "reviews") \
    .schema(schema) \
    .load() \
    .drop("_id")

25/07/06 14:50:52 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
25/07/06 14:50:52 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:978)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [10]:
df.show(2)

+---------+--------------------------------+----------+-----+------+-------------+------+------+
|  book_id|                         content|      date|likes|rating|  review_type| title|  user|
+---------+--------------------------------+----------+-----+------+-------------+------+------+
|361016676|일곱 작품 모두 몰입의 대 환장...|2025-04-09|   30|     5|CommentReview|혼모노|은하수|
|361016676|분명 각 작품은 지루하지 않게 ...|2025-05-25|   28|     5|CommentReview|혼모노|초록비|
+---------+--------------------------------+----------+-----+------+-------------+------+------+
only showing top 2 rows



25/07/06 14:49:01 ERROR StandaloneSchedulerBackend: Application has been killed. Reason: Master removed our application: KILLED
25/07/06 14:49:01 ERROR Inbox: Ignoring error
org.apache.spark.SparkException: Exiting due to error from cluster scheduler: Master removed our application: KILLED
	at org.apache.spark.errors.SparkCoreErrors$.clusterSchedulerError(SparkCoreErrors.scala:291)
	at org.apache.spark.scheduler.TaskSchedulerImpl.error(TaskSchedulerImpl.scala:978)
	at org.apache.spark.scheduler.cluster.StandaloneSchedulerBackend.dead(StandaloneSchedulerBackend.scala:165)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint.markDead(StandaloneAppClient.scala:263)
	at org.apache.spark.deploy.client.StandaloneAppClient$ClientEndpoint$$anonfun$receive$1.applyOrElse(StandaloneAppClient.scala:170)
	at org.apache.spark.rpc.netty.Inbox.$anonfun$process$1(Inbox.scala:115)
	at org.apache.spark.rpc.netty.Inbox.safelyCall(Inbox.scala:213)
	at org.apache.spark.rpc.netty.Inbox.proce

In [3]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, DoubleType, ArrayType, MapType, Row
from pyspark.sql.functions import col, lit, upper, count, avg, sum, min, max, expr, countDistinct

# SparkSession 생성 (pyspark 셸에서는 'spark' 변수로 자동 생성됨)
# spark = SparkSession.builder.appName("DataFrameAPIExamples").master("local[*]").getOrCreate()

# 예제용 데이터 및 스키마
data = [("Alice", 34, "New York", ["reading", "hiking"]),
        ("Bob", 45, "Chicago", ["music", "movies"]),
        ("Charlie", 29, "New York", ["hiking", "cooking"]),
        ("David", 52, "Chicago", ["reading", "travel"]),
        ("Alice", 34, "New York", ["reading", "hiking"])] # 중복 행 추가

schema = StructType([
    StructField("name", StringType(), True),
    StructField("age", IntegerType(), True),
    StructField("city", StringType(), True),
    StructField("hobbies", ArrayType(StringType()), True)
])

# 기본 DataFrame 생성
df = spark.createDataFrame(data, schema=schema)
print("--- 초기 DataFrame ---")
df.show(truncate=False)
df.printSchema()

--- 초기 DataFrame ---


+-------+---+--------+-----------------+
|name   |age|city    |hobbies          |
+-------+---+--------+-----------------+
|Alice  |34 |New York|[reading, hiking]|
|Bob    |45 |Chicago |[music, movies]  |
|Charlie|29 |New York|[hiking, cooking]|
|David  |52 |Chicago |[reading, travel]|
|Alice  |34 |New York|[reading, hiking]|
+-------+---+--------+-----------------+

root
 |-- name: string (nullable = true)
 |-- age: integer (nullable = true)
 |-- city: string (nullable = true)
 |-- hobbies: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [ ]:
from pyspark.sql.functions import count

top_review_counts = df.groupBy("book_id", "title").agg(
    count("*").alias("review_count")
).orderBy("review_count", ascending=False)

top_review_counts.show(10, truncate=False)

+---------+------------------------+------------+
|book_id  |title                   |review_count|
+---------+------------------------+------------+
|40869703 |소년이 온다             |373         |
|362863268|청춘의 독서 (특별증보판)|166         |
|25843736 |모순                    |61          |
|291370219|채식주의자 (리마스터판) |50          |
|361141292|단 한 번의 삶           |35          |
|362922298|빛과 실                 |34          |
|307692409|급류                    |27          |
|170482558|이기적 유전자           |23          |
|364959496|네가 있어서             |22          |
|365665217|안녕이라 그랬어         |22          |
+---------+------------------------+------------+
only showing top 10 rows



In [6]:
from pyspark.sql.functions import avg, sum

popular_books = df.groupBy("book_id", "title").agg(
    count("*").alias("review_count"),
    avg("rating").alias("avg_rating"),
    sum("likes").alias("total_likes")
).orderBy("avg_rating", ascending=False)

popular_books.show(10, truncate=False)

+---------+----------------------------------------------------------------------+------------+----------+-----------+
|book_id  |title                                                                 |review_count|avg_rating|total_likes|
+---------+----------------------------------------------------------------------+------------+----------+-----------+
|363870592|열네 살, 한국에 왔어요                                                |3           |5.0       |3          |
|353981757|2025 큰별쌤 최태성의 별★별한국사 한국사능력검정시험 심화(1, 2, 3급) 상|3           |5.0       |6          |
|330109271|ETS 토익 정기시험 기출문제집 1000 Vol. 4 Listening (리스닝)           |6           |5.0       |16         |
|366791947|아무튼, 리코더                                                        |1           |5.0       |0          |
|366270620|고전이 답했다 마땅히 가져야 할 부에 대하여                            |16          |5.0       |1          |
|365713027|저속노화 마인드셋                                                     |2           |5.0       |0          |
|36633165

In [7]:
from pyspark.sql.functions import to_date, count, col

# 날짜 형식 변환
reviews_df = df.withColumn("review_date", to_date("date"))

# 특정 도서만 필터링
target_book_title = "혼모노"  # 원하는 책 이름 입력

trend = reviews_df.groupBy("title", "review_date") \
    .agg(count("*").alias("daily_reviews")) \
    .orderBy("review_date")

# 특정 도서 필터링해서 보기
trend.filter(col("title").isin("혼모노", "불변의 법칙")) \
    .orderBy("title", "review_date") \
    .show(truncate=False)

+------+-----------+-------------+
|title |review_date|daily_reviews|
+------+-----------+-------------+
|혼모노|2025-03-27 |1            |
|혼모노|2025-03-29 |1            |
|혼모노|2025-04-04 |1            |
|혼모노|2025-04-06 |2            |
|혼모노|2025-04-07 |1            |
|혼모노|2025-04-09 |1            |
|혼모노|2025-04-14 |2            |
|혼모노|2025-04-28 |1            |
|혼모노|2025-04-30 |1            |
|혼모노|2025-05-16 |1            |
|혼모노|2025-05-22 |1            |
|혼모노|2025-05-25 |1            |
|혼모노|2025-05-28 |1            |
|혼모노|2025-06-14 |1            |
|혼모노|2025-07-01 |1            |
|혼모노|2025-07-02 |2            |
+------+-----------+-------------+



In [5]:
top_reviews = df.select("title", "user", "likes", "rating", "content") \
    .orderBy("likes", ascending=False)

top_reviews.show(10, truncate=False)

+-----------------------+-----------+-----+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|title                  |user       |likes|rating|content                                                                                                                                                                                                                                                                               |
+-----------------------+-----------+-----+------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|소년이 온다   